In [46]:
import tensorflow_hub as hub
import pandas
from sklearn.metrics.pairwise import linear_kernel

In [28]:
%time model = hub.load("/Users/Alexey.Zhebel/Downloads/universal-sentence-encoder_4")

CPU times: user 3.17 s, sys: 815 ms, total: 3.98 s
Wall time: 4.14 s


In [29]:
def embed(input):
    return model(input)

In [6]:
SentMessage =['How old are you?','what is your age?','how are you?','how you doing?']

In [7]:
Model_USE = embed(SentMessage)

In [19]:
query = ["when were you born"]

In [20]:
Q_Train = embed(query)

In [21]:
sims = linear_kernel(Q_Train, Model_USE).flatten()

In [22]:
top_index_doc = sims.argsort()[:-11:-1]

In [23]:
top_index_doc

array([0, 1, 2, 3])

In [24]:
import csv

In [25]:
print("Getting list of topic names and texts...")
with open("01_topic_text.csv") as file:
    topic_names = []
    topic_texts = []
    for line in csv.reader(file):
        topic_names.append(line[0])
        topic_texts.append(line[1])
print("Done!")

Getting list of topic names and texts...
Done!


In [26]:
topic_id = 1
print(f"Topic {topic_id} is:", topic_names[topic_id])
print(topic_texts[topic_id][:200] + "...")

Topic 1 is: configuring-project-and-ide-settings.html
Configuring the IDE
IntelliJ IDEA allows you to configure the settings on several levels: the module level, the project level, and globally.
Global settings apply to all projects that you open with a ...


In [30]:
corpus = embed(topic_texts)

In [39]:
query = embed(["file templates"])

In [40]:
sims = linear_kernel(corpus, query).flatten()

In [49]:
top_index_doc = sims.argsort()[:-6:-1]

In [50]:
top_index_doc

array([ 418,  899, 1256, 1024,  494])

In [45]:
topic_names[1256]

'using-file-and-code-templates.html'

In [52]:
results_csv = "search_test_results.csv"

with open(results_csv, "r") as file:
    df = pandas.read_csv(file)

search_queries = df["query"].tolist()

all_results = []

for query in search_queries:
    query_embedding = embed([query])
    sims = linear_kernel(corpus, query_embedding).flatten()
    top_index_doc = sims.argsort()[:-6:-1]

    q_result = ""

    for idx in top_index_doc:
        q_result = q_result + topic_names[idx] + "\n"

    all_results.append(q_result)

# print(all_results)
df["google_use_dan"] = all_results

In [54]:
df.to_csv(results_csv, index=False)